In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, AncillaRegister, IBMQ, Aer, __qiskit_version__
from qiskit.circuit.library import QFT, GraphState, GroverOperator
from qiskit.qasm import pi

import numpy as np
import matplotlib.pyplot as plt

from src.utils import *
from src.benchmarks import grover, qaoa, ghz, qft, qftentangled, shor, wstate, qpeinexact, qpeexact, graphstate, vqe, deutschjozsa, hhl, ae, qwalk

In [ ]:
provider = IBMQ.load_account()
backend = provider.backend.ibmq_santiago
c_map = backend.configuration().coupling_map

#from qiskit.test.mock import FakeMontreal
#fake_backend = FakeMontreal()
#c_map = fake_backend.configuration().coupling_map

max_n_arch = max(max(c_map))

In [ ]:
ibm_native_gates = backend.configuration().basis_gates
rigetti_native_gates = ['rx','rz','cz']

In [ ]:
benchmarks = [graphstate, grover, qaoa, ghz, qft, wstate, qpeinexact, qpeexact, vqe, deutschjozsa, hhl, shor, ae, qwalk]

In [ ]:
n_min = 3
n_max = 6
opt_level = 2
num_qubits_test = 5
save_hist = True
simulator = Aer.get_backend('qasm_simulator')
gate_set_name = "ibm"

In [ ]:
characteristics = []

if (gate_set_name == "ibm"):
    basis_gates = ibm_native_gates
elif (gate_set_name == "rigetti"):
    basis_gates = rigetti_native_gates
else:
    raise ValueError("Gate Set Error")

for benchmark in benchmarks:
    if benchmark == shor or benchmark == (hhl):
        continue
    for n in range(n_min, n_max):  
        qc = benchmark.create_circuit(n)
        
        filename = qc.name + "_algorithm_" + str(n) 
        
        serialize_qc(qc, n, filename)          
        save_circ(qc, filename)
        
        compiled_without_architecure = get_compiled_circuit(qc = qc, opt_level=opt_level, basis_gates=basis_gates)
        filename_transpiled = qc.name + "_transpiled_" + gate_set_name + "_opt" + str(opt_level) + "_"+ str(n) 
        save_as_qasm(compiled_without_architecure, n, filename_transpiled , basis_gates, opt_level)       
        
        depth = compiled_without_architecure.depth()
        
        characteristics.append([filename_transpiled, n, depth]) 
        save_circ(compiled_without_architecure, filename_transpiled)   
        
        if save_hist:
            sim_and_print_hist(compiled_without_architecure, simulator, filename_transpiled)
        
        if (len(compiled_without_architecure.qubits) < max_n_arch):
            compiled_with_architecture = get_compiled_circuit(qc = qc, opt_level=opt_level, 
                                                              basis_gates=basis_gates, c_map=c_map)
            filename_mapped = qc.name + "_mapped_" + gate_set_name + "_opt" + str(opt_level) + "_"+ str(n) 
            save_as_qasm(compiled_with_architecture, n, filename_mapped, basis_gates, 
                         opt_level, True, c_map, backend.name() if backend.name() else "")
            
            depth = compiled_with_architecture.depth()
            characteristics.append([filename_mapped, n, depth])
            save_circ(compiled_with_architecture, filename_mapped)

            if save_hist:
                sim_and_print_hist(compiled_with_architecture, simulator, filename_mapped)
                
                
## !Todo: both shor and hhl scripts needs to be adjusted to chances to this cell
#create_shor_benchmarks()
#create_hhl_benchmarks()

In [ ]:
characteristics = np.array(characteristics)
plt.scatter(x=characteristics[:,1].astype(int), y=characteristics[:,2].astype(int))
plt.yscale('log')
plt.title('Depth and Width of generated Benchmarks')
plt.xlabel('# of Qubits')
plt.ylabel('Circuit Depth');

In [ ]:
def create_shor_benchmarks():
    small = [15,4] #18 qubits
    medium = [821, 4] #42 qubits
    large = [11777, 4] #58 qubits
    xlarge = [201209, 4] # 74 qubits
    benchmark_list = [small]#, medium, large, xlarge]
    for n, a in benchmark_list:         
        qc = shor.create_circuit(n,a)
        num_qubits = len(qc.qubits)
        
        serialize_qc(qc, num_qubits) 
        
        filename = qc.name + str(n) 
        save_circ(qc, filename)
        
        compiled_without_architecure = get_compiled_circuit(qc = qc, opt_level=opt_level, basis_gates=basis_gates)
        num_qubits = len(compiled_without_architecure.qubits)
        save_as_qasm(compiled_without_architecure, num_qubits, basis_gates) 
        
        filename = qc.name + "_transpiled_" + str(num_qubits)
        depth = compiled_without_architecure.depth()        
        characteristics.append([filename, num_qubits, depth])
        save_circ(compiled_without_architecure, filename)   
        
        if save_hist:
            sim_and_print_hist(compiled_without_architecure, simulator, filename)
        
        if (num_qubits < max_n_arch):
            compiled_with_architecture = get_compiled_circuit(qc = qc, opt_level=opt_level,
                                                              basis_gates=basis_gates, c_map=c_map)
            save_as_qasm(compiled_with_architecture, num_qubits, basis_gates, True, c_map, 
                         backend.name() if backend.name() else "")
            
            filename = qc.name + "_transpiled_mapped_" + str(num_qubits)
            depth = compiled_with_architecture.depth()
            characteristics.append([filename, num_qubits, depth])
            save_circ(compiled_with_architecture, filename)
            
            if save_hist:
                sim_and_print_hist(compiled_with_architecture, simulator, filename)

In [ ]:
def create_hhl_benchmarks():
    # n is not the number of qubits in this case
    for n in range(1,4):         
        qc = hhl.create_circuit(n)
        
        num_qubits = len(qc.qubits)
        
        serialize_qc(qc, num_qubits) 
        compiled_without_architecure = get_compiled_circuit(qc = qc, opt_level=opt_level, basis_gates=basis_gates)
        num_qubits = len(compiled_without_architecure.qubits)
        save_as_qasm(compiled_without_architecure, num_qubits)        
        
        filename = qc.name + "_transpiled_" + str(num_qubits)
        depth = compiled_without_architecure.depth()        
        characteristics.append([filename, num_qubits, depth])
        
        if print_hist:
            sim_and_print_hist(compiled_without_architecure, simulator, filename)
        
        if (num_qubits < max_n_arch):
            compiled_with_architecture = get_compiled_circuit(qc = qc, opt_level=opt_level, 
                                                              basis_gates=basis_gates, c_map=c_map)
            save_as_qasm(compiled_with_architecture, num_qubits, True)
            
            filename = qc.name + "_transpiled_mapped_" + str(num_qubits)
            depth = compiled_with_architecture.depth()
            characteristics.append([filename, num_qubits, depth])
            
            if print_hist:
                sim_and_print_hist(compiled_with_architecture, simulator, filename)

Grover Test

In [ ]:
a = grover.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2, basis_gates=basis_gates)
save_as_qasm(b, num_qubits_test, basis_gates)
c = get_compiled_circuit(b, opt_level=2, basis_gates=basis_gates, c_map=c_map)
save_as_qasm(c, num_qubits_test, basis_gates, True, c_map, backend.name() if backend.name() else "")

QAOA Test

In [ ]:
a = qaoa.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2, basis_gates=basis_gates)
save_as_qasm(b, num_qubits_test, basis_gates)
c = get_compiled_circuit(b, opt_level=2, basis_gates=basis_gates, c_map=c_map)
save_as_qasm(c, num_qubits_test, basis_gates, True, c_map, backend.name() if backend.name() else "")

GHZ Test

In [ ]:
a = ghz.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2, basis_gates=basis_gates)
save_as_qasm(b, num_qubits_test, basis_gates)
c = get_compiled_circuit(b, opt_level=2, basis_gates=basis_gates, c_map=c_map)
save_as_qasm(c, num_qubits_test, basis_gates, True, c_map, backend.name() if backend.name() else "")

QFT Test

In [ ]:
a = qft.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2, basis_gates=basis_gates)
save_as_qasm(b, num_qubits_test, basis_gates)
c = get_compiled_circuit(b, opt_level=2, basis_gates=basis_gates, c_map=c_map)
save_as_qasm(c, num_qubits_test, basis_gates, True, c_map, backend.name() if backend.name() else "")

QFT Entangled Test

In [ ]:
a = qft_entangled.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2, basis_gates=basis_gates)
save_as_qasm(b, num_qubits_test, basis_gates)
c = get_compiled_circuit(b, opt_level=2, basis_gates=basis_gates, c_map=c_map)
save_as_qasm(c, num_qubits_test, basis_gates, True, c_map, backend.name() if backend.name() else "")

W-State Test

In [ ]:
a = w_state.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2, basis_gates=basis_gates)
save_as_qasm(b, num_qubits_test, basis_gates)
c = get_compiled_circuit(b, opt_level=2, basis_gates=basis_gates, c_map=c_map)
save_as_qasm(c, num_qubits_test, basis_gates, True, c_map, backend.name() if backend.name() else "")

QPE Exact and QPE Inexact Test

In [ ]:
a = qpe_exact.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2, basis_gates=basis_gates)
save_as_qasm(b, num_qubits_test, basis_gates)
c = get_compiled_circuit(b, opt_level=2, basis_gates=basis_gates, c_map=c_map)
save_as_qasm(c, num_qubits_test, basis_gates, True, c_map, backend.name() if backend.name() else "")

a = qpe_inexact.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2, basis_gates=basis_gates)
save_as_qasm(b, num_qubits_test, basis_gates)
c = get_compiled_circuit(b, opt_level=2, basis_gates=basis_gates, c_map=c_map)
save_as_qasm(c, num_qubits_test, basis_gates, True, c_map, backend.name() if backend.name() else "")

Graph State Test

In [ ]:
a = graph_state.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2, basis_gates=basis_gates)
save_as_qasm(b, num_qubits_test, basis_gates)
c = get_compiled_circuit(b, opt_level=2, basis_gates=basis_gates, c_map=c_map)
save_as_qasm(c, num_qubits_test, basis_gates, True, c_map, backend.name() if backend.name() else "")


Shor Test

In [ ]:
a = shor.create_circuit(15)
b = get_compiled_circuit(a, opt_level=2, basis_gates=basis_gates)
save_as_qasm(b, None, basis_gates)
#c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
#save_as_qasm(c, None, True)

AE Test

In [ ]:
a = ae.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2, basis_gates=basis_gates)
save_as_qasm(b, num_qubits_test, basis_gates)
c = get_compiled_circuit(b, opt_level=2, basis_gates=basis_gates, c_map=c_map)
save_as_qasm(c, num_qubits_test, basis_gates, True, c_map, backend.name() if backend.name() else "")

DJ Test

In [ ]:
num_qubits_test = 5
a = deutsch_jozsa.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2, basis_gates=basis_gates)
save_as_qasm(b, num_qubits_test, basis_gates)
c = get_compiled_circuit(b, opt_level=2, basis_gates=basis_gates, c_map=c_map)
save_as_qasm(c, num_qubits_test, basis_gates, True, c_map, backend.name() if backend.name() else "")

HHL Test

In [ ]:
import src.benchmarks.hhl
num_qubits_test = 3
a = hhl.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2, basis_gates=basis_gates)
save_as_qasm(b, num_qubits_test, basis_gates)
c = get_compiled_circuit(b, opt_level=2, basis_gates=basis_gates, c_map=c_map)
save_as_qasm(c, num_qubits_test, basis_gates, True, c_map, backend.name() if backend.name() else "")

Example Generation for a figure in the paper

In [ ]:
#from qiskit.circuit.library import RealAmplitudes
#from qiskit.visualization import plot_histogram, plot_gate_map, plot_circuit_layout
#plot_gate_map(backend).savefig("layout_santiago.png")
#ansatz = RealAmplitudes(3, reps=1)
#qc = ansatz.decompose()
#qc.draw(output="mpl", filename="paper_example_1_1")
#b = get_compiled_circuit(qc, opt_level=0, basis_gates=basis_gates)
#b.draw(output="mpl", filename="paper_example_1_2")
#c = get_compiled_circuit(b, opt_level=0, basis_gates=basis_gates, c_map=c_map)
#c.draw(output="mpl", filename="paper_example_1_3")

Qiskit Finance Test

In [ ]:

from src.benchmarks.qiskit_application_finance import portfolio_optimization_qaoa, portfolio_optimization_vqe, pricing_eur_call_opt, pricing_eur_put_opt
num_qubits_test = 5
a = pricing_eur_put_opt.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2, basis_gates=basis_gates)
save_as_qasm(b, num_qubits_test, basis_gates)
c = get_compiled_circuit(b, opt_level=2, basis_gates=basis_gates, c_map=c_map)
save_as_qasm(c, num_qubits_test, basis_gates, True, c_map, backend.name() if backend.name() else "")

Qiskit Optimization Test

In [ ]:

from src.benchmarks.qiskit_application_optimization import vehicle_routing
num_qubits_test = 5
a = vehicle_routing.create_circuit(3,3)
#b = get_compiled_circuit(a, opt_level=2)
#save_as_qasm(b, num_qubits_test)
#c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
#save_as_qasm(c, num_qubits_test, True)

Qiskit Machine Learning Test

In [ ]:
import numpy as np
seed = 71
np.random.seed = seed

import matplotlib.pyplot as plt
%matplotlib inline

from qiskit import QuantumRegister, QuantumCircuit, BasicAer
from qiskit.circuit.library import TwoLocal, UniformDistribution

from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit_machine_learning.algorithms import NumPyDiscriminator, QGAN

algorithm_globals.random_seed = seed

# Number training data samples
N = 1000

# Load data samples from log-normal distribution with mean=1 and standard deviation=1
mu = 1
sigma = 1
real_data = np.random.lognormal(mean=mu, sigma=sigma, size=N)

# Set the data resolution
# Set upper and lower data values as list of k min/max data values [[min_0,max_0],...,[min_k-1,max_k-1]]
bounds = np.array([0., 3.])
# Set number of qubits per data dimension as list of k qubit values[#q_0,...,#q_k-1]
num_qubits = [3]
k = len(num_qubits)

In [ ]:
# Set number of training epochs
# Note: The algorithm's runtime can be shortened by reducing the number of training epochs.
num_epochs = 10
# Batch size
batch_size = 100

# Initialize qGAN
qgan = QGAN(real_data, bounds, num_qubits, batch_size, num_epochs, snapshot_dir=None)
qgan.seed = 1
# Set quantum instance to run the quantum generator
quantum_instance = QuantumInstance(
    backend=BasicAer.get_backend("statevector_simulator"), seed_transpiler=seed, seed_simulator=seed
)

# Set entangler map
#entangler_map = [[0, 1]]


# Set an initial state for the generator circuit
init_dist = UniformDistribution(sum(num_qubits))

# Set the ansatz circuit
ansatz = TwoLocal(int(np.sum(num_qubits)), "ry", "cz",  reps=1) #entanglement=entangler_map,

# Set generator's initial parameters - in order to reduce the training time and hence the
# total running time for this notebook
#init_params = [3.0, 1.0, 0.6, 1.6]

# You can increase the number of training epochs and use random initial parameters.
init_params = np.random.rand(ansatz.num_parameters_settable) * 2 * np.pi

# Set generator circuit by adding the initial distribution infront of the ansatz
g_circuit = ansatz.compose(init_dist, front=True)

# Set quantum generator
qgan.set_generator(generator_circuit=g_circuit, generator_init_params=init_params)
# The parameters have an order issue that following is a temp. workaround
qgan._generator._free_parameters = sorted(g_circuit.parameters, key=lambda p: p.name)
# Set classical discriminator neural network
discriminator = NumPyDiscriminator(len(num_qubits))
qgan.set_discriminator(discriminator)

In [ ]:
# Run qGAN
result = qgan.run(quantum_instance)

In [ ]:
params = qgan.generator.parameter_values

In [ ]:
params = qgan.generator.parameter_values
qc = qgan.generator.construct_circuit(params=params)
qc.draw()

In [ ]:
qgan.generator.generator_circuit.draw()

In [ ]:
#Plot the CDF of the resulting distribution against the target distribution, i.e. log-normal
log_normal = np.random.lognormal(mean=1, sigma=1, size=100000)
log_normal = np.round(log_normal)
log_normal = log_normal[log_normal <= bounds[1]]
temp = []
for i in range(int(bounds[1] + 1)):
    temp += [np.sum(log_normal==i)]
log_normal = np.array(temp / sum(temp))

plt.figure(figsize=(6,5))
plt.title('CDF (Cumulative Distribution Function)')
samples_g, prob_g = qgan.generator.get_output(qgan.quantum_instance, shots=10000)
samples_g = np.array(samples_g)
samples_g = samples_g.flatten()
num_bins = len(prob_g)
plt.bar(samples_g,  np.cumsum(prob_g), color='royalblue', width= 0.8, label='simulation')
plt.plot( np.cumsum(log_normal),'-o', label='log-normal', color='deepskyblue', linewidth=4, markersize=12)
plt.xticks(np.arange(min(samples_g), max(samples_g)+1, 1.0))
plt.grid()
plt.xlabel('x')
plt.ylabel('p(x)')
plt.legend(loc='best')
plt.show()

AE Test

In [ ]:
from src.benchmarks import canonical_ae


In [ ]:
from qiskit import BasicAer
from qiskit.utils import QuantumInstance

backend = BasicAer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend)

In [ ]:
a = canonical_ae.create_circuit(5)
a.draw()

Rigetti Coupling Map Creation

In [ ]:
c_map_rigetti = []
for j in range(4):
    for i in range(0,7):
        elem = [i+j*10, i+1+j*10]
        c_map_rigetti.append(elem)
        if i == 6:
            c_map_rigetti.append([0+j*10, 7+j*10])
            
# for i in range(10,17):
#     elem = [i, i+1]
#     c_map_rigetti.append(elem)
# for i in range(20,27):
#     elem = [i, i+1]
#     c_map_rigetti.append(elem)
# for i in range(20,27):
#     elem = [i, i+1]
#     c_map_rigetti.append(elem)
# for i in range(30,37):
#     elem = [i, i+1]
#     c_map_rigetti.append(elem)    
c_map_rigetti.extend(([2,15],[1,16],[12,25],[11,26],[22,35],[21,36]))    
print(c_map_rigetti)

In [ ]:
import networkx as nx
G = nx.Graph()
G.add_edges_from(c_map_rigetti)
nx.draw(G, with_labels=True)

Google Coupling Map Creation

In [ ]:
c_map_google = []
for j in range(8):
    for i in range(6):
        if j%2 == 0:
            if i != 0:
                elem = [i+6*j, i+6*j + 5]
                c_map_google.append(elem)

            if i != 6:
                elem = [i+6*j, i+6*j + 6]
                c_map_google.append(elem)

        else:
            if i != 0:
                elem = [i+6*j, i+6*j + 6]
                c_map_google.append(elem)

            if i != 6:
                elem = [i+6*j, i+6*j + 7]
                c_map_google.append(elem)
print(c_map_google)

In [ ]:
G = nx.Graph()
G.add_edges_from(c_map_google)
nx.draw(G, with_labels=True)

TSP and vehicle routing Test

In [ ]:
from src.benchmarks.qiskit_application_optimization import travelling_salesman, vehicle_routing

In [ ]:
qc = vehicle_routing.create_circuit(4,1)

In [ ]:
qc1, qp1 = vehicle_routing.create_circuit(4,2)

In [ ]:
qc2, qp2  = vehicle_routing.create_circuit(4,3)

In [ ]:
qc3, qp3  = vehicle_routing.create_circuit(4,4)

In [ ]:
from qiskit.quantum_info import Statevector
print(Statevector.from_instruction(qc1).equiv(Statevector.from_instruction(qc2)),
Statevector.from_instruction(qc1).equiv(Statevector.from_instruction(qc3)),
Statevector.from_instruction(qc3).equiv(Statevector.from_instruction(qc2)))

QGAN Test

In [ ]:
from src.benchmarks.qiskit_application_ml import qgan

In [ ]:
a = qgan.create_circuit(4)

In [ ]:
a.decompose().draw()

In [ ]:
qc.draw()

Qiskit Application Finance Test

In [1]:
from src.benchmarks.qiskit_application_finance import portfolio_optimization_qaoa, portfolio_optimization_vqe, pricing_eur_call_opt, pricing_eur_put_opt

C:\Users\Nils\anaconda3\envs\qsharp-env\lib\site-packages\qiskit\aqua\__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')
C:\Users\Nils\anaconda3\envs\qsharp-env\lib\site-packages\qiskit\finance\__init__.py:50: DeprecationWarning: The package qiskit.finance is deprecated. It was moved/refactored to qiskit_finance (pip install qiskit-finance). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('finance', 'qiskit_finance', 'qiskit-finance')
C:\Users\Nils\anaconda3\envs\qsharp-env\lib\site-packages\qiskit\optimization\__init__.py:92: DeprecationWarning: The package qiskit.optimization is deprecated. It was moved/refactored to qiskit_optimization (pip install qiskit-optimization). For more information see <https://github.com/Q

In [ ]:
b = portfolio_optimization_qaoa.create_circuit(5)

In [ ]:
b.draw()

In [ ]:
c = portfolio_optimization_vqe.create_circuit(4)
c.draw()

In [ ]:
d = pricing_eur_call_opt.create_circuit(4)
d.draw()

In [2]:
e, iae = pricing_eur_put_opt.create_circuit(2)
e.draw()

C:\Users\Nils\anaconda3\envs\qsharp-env\lib\site-packages\qiskit\aqua\algorithms\amplitude_estimators\ae_algorithm.py:91: DeprecationWarning: The package qiskit.aqua.algorithms.amplitude_estimators is deprecated. It was moved/refactored to qiskit.algorithms.amplitude_estimators (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua.algorithms.amplitude_estimators',


┌────────────────────────┐┌────┐┌────┐
q_0: ─────────────────────────┤0                       ├┤0   ├┤0   ├
     ┌───────────────────────┐│  multiplex2_reverse_dg ││    ││    │
q_1: ┤ multiplex1_reverse_dg ├┤1                       ├┤1   ├┤1   ├
     └───────────────────────┘└────────────────────────┘│    ││    │
q_2: ───────────────────────────────────────────────────┤2 F ├┤2 Q ├
                                                        │    ││    │
q_3: ───────────────────────────────────────────────────┤3   ├┤3   ├
                                                        │    ││    │
q_4: ───────────────────────────────────────────────────┤4   ├┤4   ├
                                                        └────┘└────┘

In [16]:
test1 = iae.construct_circuit(1)
test1.decompose().decompose().decompose().draw()

global phase: π
     ┌───────────────┐ ┌───┐┌───────────────┐┌───┐┌─────────┐        ┌──────┐»
q_0: ┤ R(1.7377,π/2) ├─┤ X ├┤ R(1.3166,π/2) ├┤ X ├┤0        ├────────┤0     ├»
     ├───────────────┴┐└─┬─┘└───────────────┘└─┬─┘│         │        │      │»
q_1: ┤ U3(1.4598,0,0) ├──■─────────────────────■──┤1 LinRot ├────────┤1     ├»
     └────────────────┘                           │         │┌──────┐│      │»
q_2: ─────────────────────────────────────────────┤2        ├┤ P(0) ├┤  cmp ├»
                                                  └─────────┘└──────┘│      │»
q_3: ────────────────────────────────────────────────────────────────┤2     ├»
                                                                     │      │»
q_4: ────────────────────────────────────────────────────────────────┤3     ├»
                                                                     └──────┘»
«                                                                 »
«q_0: ────────────────────────────────────────────────────────────»
«                                                                 »
«q_1: ────────────────────────────────────────────────────────────»
«             ┌───┐┌────────────┐┌───┐┌─────────────┐┌──────┐┌───┐»
«q_2: ────────┤ X ├┤ U(π/8,0,0) ├┤ X ├┤ U(-π/8,0,0) ├┤ P(0) ├┤ X ├»
«     ┌──────┐└─┬─┘└────────────┘└─┬─┘└───┬──────┬──┘└──────┘└─┬─┘»
«q_3: ┤ P(0) ├──■──────────────────■──────┤ P(0) ├─────────────■──»
«     └──────┘                            └──────┘                »
«q_4: ────────────────────────────────────────────────────────────»
«                                                                 »
«                                                                 »
«q_0: ────────────────────────────────────────────■───────────────»
«                                                 │               »
«q_1: ────────────────────────────────────────────┼───────────────»
«     ┌─────────────────┐┌───┐┌────────────────┐┌─┴─┐┌──────┐┌───┐»
«q_2: ┤ U(-0.11449,0,0) ├┤ X ├┤ U(0.11449,0,0) ├┤ X ├┤ P(0) ├┤ X ├»
«     └─────────────────┘└─┬─┘└────────────────┘└─┬─┘├──────┤└─┬─┘»
«q_3: ─────────────────────■──────────────────────■──┤ P(0) ├──■──»
«                                                    └──────┘     »
«q_4: ────────────────────────────────────────────────────────────»
«                                                                 »
«                                                                 »
«q_0: ────────────────────────────────────────────■───────────────»
«                                                 │               »
«q_1: ────────────────────────────────────────────┼───────────────»
«     ┌────────────────┐┌───┐┌─────────────────┐┌─┴─┐┌──────┐┌───┐»
«q_2: ┤ U(0.11449,0,0) ├┤ X ├┤ U(-0.11449,0,0) ├┤ X ├┤ P(0) ├┤ X ├»
«     └────────────────┘└─┬─┘└─────────────────┘└─┬─┘├──────┤└─┬─┘»
«q_3: ────────────────────■───────────────────────■──┤ P(0) ├──■──»
«                                                    └──────┘     »
«q_4: ────────────────────────────────────────────────────────────»
«                                                                 »
«                                                                 »
«q_0: ────────────────────────────────────────────────────────────»
«                                                                 »
«q_1: ────────────────────────────────────────────■───────────────»
«     ┌─────────────────┐┌───┐┌────────────────┐┌─┴─┐┌──────┐┌───┐»
«q_2: ┤ U(-0.22898,0,0) ├┤ X ├┤ U(0.22898,0,0) ├┤ X ├┤ P(0) ├┤ X ├»
«     └─────────────────┘└─┬─┘└────────────────┘└─┬─┘├──────┤└─┬─┘»
«q_3: ─────────────────────■──────────────────────■──┤ P(0) ├──■──»
«                                                    └──────┘     »
«q_4: ────────────────────────────────────────────────────────────»
«                                                                 »
«                                                    ┌─────────┐           »
«q_0: ────────────────────────────────────────

In [17]:
test2 = iae.construct_circuit(2)
test2.decompose().decompose().decompose().draw()

┌───────────────┐ ┌───┐┌───────────────┐┌───┐┌─────────┐        ┌──────┐»
q_0: ┤ R(1.7377,π/2) ├─┤ X ├┤ R(1.3166,π/2) ├┤ X ├┤0        ├────────┤0     ├»
     ├───────────────┴┐└─┬─┘└───────────────┘└─┬─┘│         │        │      │»
q_1: ┤ U3(1.4598,0,0) ├──■─────────────────────■──┤1 LinRot ├────────┤1     ├»
     └────────────────┘                           │         │┌──────┐│      │»
q_2: ─────────────────────────────────────────────┤2        ├┤ P(0) ├┤  cmp ├»
                                                  └─────────┘└──────┘│      │»
q_3: ────────────────────────────────────────────────────────────────┤2     ├»
                                                                     │      │»
q_4: ────────────────────────────────────────────────────────────────┤3     ├»
                                                                     └──────┘»
«                                                                 »
«q_0: ────────────────────────────────────────────────────────────»
«                                                                 »
«q_1: ────────────────────────────────────────────────────────────»
«             ┌───┐┌────────────┐┌───┐┌─────────────┐┌──────┐┌───┐»
«q_2: ────────┤ X ├┤ U(π/8,0,0) ├┤ X ├┤ U(-π/8,0,0) ├┤ P(0) ├┤ X ├»
«     ┌──────┐└─┬─┘└────────────┘└─┬─┘└───┬──────┬──┘└──────┘└─┬─┘»
«q_3: ┤ P(0) ├──■──────────────────■──────┤ P(0) ├─────────────■──»
«     └──────┘                            └──────┘                »
«q_4: ────────────────────────────────────────────────────────────»
«                                                                 »
«                                                                 »
«q_0: ────────────────────────────────────────────■───────────────»
«                                                 │               »
«q_1: ────────────────────────────────────────────┼───────────────»
«     ┌─────────────────┐┌───┐┌────────────────┐┌─┴─┐┌──────┐┌───┐»
«q_2: ┤ U(-0.11449,0,0) ├┤ X ├┤ U(0.11449,0,0) ├┤ X ├┤ P(0) ├┤ X ├»
«     └─────────────────┘└─┬─┘└────────────────┘└─┬─┘├──────┤└─┬─┘»
«q_3: ─────────────────────■──────────────────────■──┤ P(0) ├──■──»
«                                                    └──────┘     »
«q_4: ────────────────────────────────────────────────────────────»
«                                                                 »
«                                                                 »
«q_0: ────────────────────────────────────────────■───────────────»
«                                                 │               »
«q_1: ────────────────────────────────────────────┼───────────────»
«     ┌────────────────┐┌───┐┌─────────────────┐┌─┴─┐┌──────┐┌───┐»
«q_2: ┤ U(0.11449,0,0) ├┤ X ├┤ U(-0.11449,0,0) ├┤ X ├┤ P(0) ├┤ X ├»
«     └────────────────┘└─┬─┘└─────────────────┘└─┬─┘├──────┤└─┬─┘»
«q_3: ────────────────────■───────────────────────■──┤ P(0) ├──■──»
«                                                    └──────┘     »
«q_4: ────────────────────────────────────────────────────────────»
«                                                                 »
«                                                                 »
«q_0: ────────────────────────────────────────────────────────────»
«                                                                 »
«q_1: ────────────────────────────────────────────■───────────────»
«     ┌─────────────────┐┌───┐┌────────────────┐┌─┴─┐┌──────┐┌───┐»
«q_2: ┤ U(-0.22898,0,0) ├┤ X ├┤ U(0.22898,0,0) ├┤ X ├┤ P(0) ├┤ X ├»
«     └─────────────────┘└─┬─┘└────────────────┘└─┬─┘├──────┤└─┬─┘»
«q_3: ─────────────────────■──────────────────────■──┤ P(0) ├──■──»
«                                                    └──────┘     »
«q_4: ────────────────────────────────────────────────────────────»
«                                                                 »
«                                                    ┌─────────┐           »
«q_0: ───────────────────────────────────────────────┤0        ├───

In [14]:
test1 == test2

False

Qiskit Application Nature Test

In [18]:
## Code from https://qiskit.org/documentation/nature/tutorials/03_ground_state_solvers.html

from qiskit_nature.drivers import Molecule
from qiskit_nature.drivers.second_quantization import (
    ElectronicStructureDriverType,
    ElectronicStructureMoleculeDriver,
)
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper

from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit_nature.algorithms import VQEUCCFactory
from qiskit_nature.algorithms import GroundStateEigensolver

molecule = Molecule(
    geometry=[["H", [0.0, 0.0, 0.0]], ["H", [0.0, 0.0, 0.735]]], charge=0, multiplicity=1
)
driver = ElectronicStructureMoleculeDriver(
    molecule, basis="sto3g", driver_type=ElectronicStructureDriverType.PYSCF
)

es_problem = ElectronicStructureProblem(driver)
qubit_converter = QubitConverter(JordanWignerMapper())

quantum_instance = QuantumInstance(backend=Aer.get_backend("aer_simulator_statevector"))
vqe_solver = VQEUCCFactory(quantum_instance)


calc = GroundStateEigensolver(qubit_converter, vqe_solver)
res = calc.solve(es_problem)

print(res)

MissingOptionalLibraryError: "The 'PySCF' library is required to use 'PySCFDriver'. You can install it with 'pip install 'qiskit-nature[pyscf]''.  See https://pyscf.org/install.html."

In [25]:
vqe_solver.get_solver(es_problem, qubit_converter).get_optimal_circuit()

AttributeError: 'NoneType' object has no attribute 'get_property'

In [22]:
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.drivers.second_quantization import (
    ElectronicStructureDriverType,
    ElectronicStructureMoleculeDriver,
)
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper

from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit_nature.algorithms import GroundStateEigensolver, QEOM, VQEUCCFactory
from qiskit_nature.algorithms import ExcitedStatesEigensolver

molecule = Molecule(
    geometry=[["H", [0.0, 0.0, 0.0]], ["H", [0.0, 0.0, 0.735]]], charge=0, multiplicity=1
)
driver = ElectronicStructureMoleculeDriver(
    molecule, basis="sto3g", driver_type=ElectronicStructureDriverType.PYSCF
)

es_problem = ElectronicStructureProblem(driver)
qubit_converter = QubitConverter(JordanWignerMapper())


# This first part sets the ground state solver
# see more about this part in the ground state calculation tutorial
quantum_instance = QuantumInstance(Aer.get_backend("aer_simulator_statevector"))
solver = VQEUCCFactory(quantum_instance)
gsc = GroundStateEigensolver(qubit_converter, solver)

# The qEOM algorithm is simply instantiated with the chosen ground state solver
qeom_excited_states_calculation = QEOM(gsc, "sd")


qeom_results = qeom_excited_states_calculation.solve(es_problem)

MissingOptionalLibraryError: "The 'PySCF' library is required to use 'PySCFDriver'. You can install it with 'pip install 'qiskit-nature[pyscf]''.  See https://pyscf.org/install.html."